In [ ]:
# 107303538_1021_hw6_2

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import csv # csv為資料儲存的型態
# csv非同質 tensor為同質

希望找到酒的quality和化學成分的關係

匯入csv資料wineq_numpy

In [ ]:
import csv # csv為資料儲存的型態
wine_path = "winequality-white.csv" # 抓酒資料庫
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1) # ( 路徑, 轉成32-float, 分隔標記, 不取第一行的字母 )
wineq_numpy # 仍為numpy array

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

拿出名稱co_list

In [ ]:
col_list = next(csv.reader(open(wine_path), delimiter=';')) # next為第一列 打開後讀並去掉; 
wineq_numpy.shape, col_list # 共有4898橫列*12直行  取col_list(也就是每個化學物質的名稱)

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

wineq_numpy轉成tensor

In [ ]:
wineq = torch.from_numpy(wineq_numpy) # 轉成tensor
wineq.shape, wineq.dtype # tensor-float32

(torch.Size([4898, 12]), torch.float32)

分離並整理化學物質和quality

取化學物質(除quality以外)(data = 化學性質)

In [ ]:
data = wineq[:, :-1] # 列都要 除了最後一行quality其他都要
data, data.shape # 總共11個化學性質

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

取quality(target = quality)

In [ ]:
target = wineq[:, -1] # 列都要 只要最後一行quality
target, target.shape # 一維 1個quality

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

target 整數化

In [ ]:
target = wineq[:, -1].long() # target變成integer
target

tensor([6, 6, 6,  ..., 6, 7, 6])

one-hot化(target_onehot為2D)

In [ ]:
target_onehot = torch.zeros(target.shape[0], 10) # 全0的tensor(4898筆(列) 10個0(行)) target.shape[0]=4898 10種分數
target_onehot.scatter_(1, target.unsqueeze(1), 1.0) # 3D的unsqueeze(1)為跳列 target.unsqueeze()為在有值的地方 填入1.0
# scatter_() = (dim, index, src) 內建為3D-tensor dim=1為變化列(自動跳列) index為變化哪一行 src為指定到的那個位置值指定為1

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

target變2D target_unsqueezed (為與target_onehot計算)

In [ ]:
target_unsqueezed = target.unsqueeze(1) # 原本target為一維[4898] unsqueeze多一維度[4898,1]
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

(略)data正規化

In [ ]:
# torch.mean的dim=0為行 
data_mean = torch.mean(data, dim=0) # 將data以行為單位平均(相同化學材料平均)
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [ ]:
# data 變異值
data_var = torch.var(data, dim=0) # 變異值(標準差的平方)
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [ ]:
# data 標準化
data_normalized = (data - data_mean) / torch.sqrt(data_var) # 標準化 sqrt為開根號
data_normalized # 結果介於-1 1

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

分類

(略)用bool判斷   ，.sum()為符合true的數量   ，儲存 = data[若為true]

In [ ]:
# step 1. 了解是用bool判斷 .sum()為符合true的數量
bad_indexes = target <= 3 # 若是target(quality)分數小於3 則為bad_indexes 
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum() # .dtype = 使用bool判斷, sum() = 只有20個為bad

(torch.Size([4898]), torch.bool, tensor(20))

In [ ]:
# step 2. advanced indexing 法, 儲存 = data[若為true]
bad_data = data[bad_indexes] # 若bad_indexs為true(為1) 則會撈到bad_data內儲存
bad_data.shape # 20個 11個data化學性質

torch.Size([20, 11])

正式分類

In [ ]:
# 分三類
bad_data = data[target <= 3] # bad 小於 3
mid_data = data[(target > 3) & (target < 7)] # mid 為 3 和 7 聯集
good_data = data[target >= 7] # good 大於 7

In [ ]:
# 三類各項平均
bad_mean = torch.mean(bad_data, dim=0) # bad 這類 data 的各項化學材料平均， 同上 
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

顯示每個target類別下 低中高的個別平均

In [ ]:
# 整理
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)): # 把多個list zip在一起
  print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

# for 中有兩個變數i args需要迭代
# enumerate為標號工具 enumerate(待標號, 起始值預設為[start=0])
# {:6.2f} 輸出小數點後面兩位的浮點數，空格6格

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


由上可知 total sulfur dioxide 可能跟酒quality有某種關係(反比)

預測total sulfur dioxide (第六行) 為good有2727個

In [ ]:
total_sulfur_threshold = 141.83 # sulfur 門檻值(以上為good 以下為bad)
total_sulfur_data = data[:,6] # 取所有的第六行
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold) # lt(a, b) 相當於 a < b, data < threshold 為 good
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum() # 預測有2727個good

(torch.Size([4898]), torch.bool, tensor(2727))

真實情況good的情況有3258個

In [ ]:
actual_indexes = target > 5 # 是good的情況 5為mid的中間數(和threshold對應)
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum() # 真實有3258個good

(torch.Size([4898]), torch.bool, tensor(3258))

只有2018個重疊，重疊部分占真實good比例為62%

In [ ]:
# 預測和真實情況的聯集
n_matches = torch.sum(actual_indexes & predicted_indexes).item() # 預測和真實情況的聯集的數量 抓item
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()
n_matches, n_matches / n_predicted, n_matches / n_actual # 只有2018個重疊，重疊部分占真實good比例為62%

(2018, 0.74000733406674, 0.6193984039287906)